In [ ]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

def fc_to_image(fc, property, scale = 1000):
    # Convert the feature collection to an image
    return fc.reduceToImage(
            properties = [property],
            reducer = ee.Reducer.first()
        ).reproject(
            crs = 'EPSG:4326',  # or match your source CRS
            scale = scale
        ).rename(property)

In [ ]:
# Absolute error
relative_growth = ee.FeatureCollection(f"{data_folder}/results/relative_growth")
relative_growth = fc_to_image(relative_growth, "percentage")

vis_params = {'min': 40, 'max': 60, 'palette': ['yellow', 'blue']}

map = geemap.Map(basemap = 'CartoDB.Positron')
map.addLayer(relative_growth, vis_params, "relative_growth")
map.add_colorbar(vis_params, label = "Percentage of asymptote", layer_name = "relative_growth")
map



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [24]:
# Absolute error
absolute_error = ee.FeatureCollection(f"{data_folder}/results/absolute_error")
absolute_error = fc_to_image(absolute_error, "error")

# Percent error
percent_error = ee.FeatureCollection(f"{data_folder}/results/percent_error")
percent_error = fc_to_image(percent_error, "error")

vis_params = {'min': -100, 'max': 100, 'palette': ['red', 'blue']}

map = geemap.Map(basemap = 'CartoDB.Positron')
map.addLayer(absolute_error, vis_params, "absolute_error")
map.addLayer(percent_error, vis_params, "percent_error")
map.add_colorbar(vis_params, label = "Absolute error", layer_name = "absolute_error")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
age = ee.Image(f"{data_folder}/mapbiomas_2020")
ESA_CCI = ee.Image(f"{data_folder}/raw/biomass").rename("biomass")
biomass = ESA_CCI.updateMask(age.gt(0)).select("biomass").reproject(crs='EPSG:4326', scale=1000)

asymptote = ee.Image(f"{data_folder}/nearest_mature").selfMask()

# Predictions for 2050
pred_lag_2050 = ee.FeatureCollection(f"{data_folder}/results/pred_lag_2050")
pred_lag_2050 = fc_to_image(pred_lag_2050, "pred_lag_2", scale=1000)

vis_params = {'min': 0, 'max': 250, 'palette': ['yellow', 'green']}

map = geemap.Map(basemap = 'CartoDB.Positron')
# map.centerObject(roi, 6)
map.addLayer(biomass, vis_params, "biomass_2020")
map.addLayer(pred_lag_2050, vis_params, "biomass_2050")
map.addLayer(asymptote, vis_params, "asymptote")
map.add_colorbar(vis_params, label = "biomass (Mg/ha)", layer_name = "biomass")
map




Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…